In [1]:
#imports
import numpy as np
import pandas as pd
import matplotlib as plt
import os


In [2]:
#set working directory
wd = os.path.dirname(os.getcwd())
os.chdir(wd)
print("Working Directory: ", os.getcwd())

Working Directory:  c:\Users\Stephan\Documents\Python\Python-Stat\Neuer\Kaggle_semiinar\student-performance


In [ ]:
labels = pd.read_csv('data/raw/train_labels.csv')

In [ ]:
labels['session'] = labels.session_id.apply(lambda x: int(x.split('_')[0]) )
labels['q'] = labels.session_id.apply(lambda x: int(x.split('_')[-1][1:]) )

In [ ]:
#deleting the now obsolete labels of the labels
labels= labels.set_index('session')
labels = labels.drop(["session_id"], axis= 1)


In [ ]:
#saving the dataset



labels.to_csv("data/processed/labels.csv",index_label= "session")

The Dataframe labels is saved. 
it contains the cols:
 session : the session id
 correct : 0 and 1 for the answer of the question
 q: number of the question from 1 to 18

The Data are stored as a .csv  

to load the data efficiently use this loading function



In [4]:
dtypes_labels= {
    'correct': np.uint8, 
    'q':np.uint8}

dataset_df = pd.read_csv('data/processed/labels.csv', dtype=dtypes_labels)